In [6]:
import numpy as np

In [7]:
f_carrier = 5000 #Frequency of the carrier signal
t_carrier = 1/F_carrier #Time of the carrier signal
sampling_freq = 22050 #sampling freq. in hz
time_per_sample = 1/sampling_freq
sample_per_symbol = sampling_freq / f_carrier
data_size = 160 * 256
n_sample = data_size * sample_per_symbol

In [8]:
def raised_cosine(Tc,t,beta=0.5) :
    mul_factor = (4*beta)/(np.pi * np.sqrt(Tc))
    numerator = np.cos((1 + beta) * np.pi * (t/Tc)) + (((1-beta)*np.pi)/4*beta) * np.sinc((1 - beta)*(t/Tc))
    denominator = 1 - (2*beta*(t/Tc)) ** 2
    return mul_factor * numerator / denominator

In [9]:
#carier generation
t = np.arange(0,Sample_per_symbol * time_per_sample, step=time_per_sample) 
pulse = raised_cosine(T_carrier,t)

In [10]:
print(pulse)

[ 53.85465057  31.96647063 -20.19726626 -70.12476185 -73.09424729]
